In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import plotly.express as px

os.chdir(r"H:\all\RL_Shrinkage_2024")

import psutil
psutil.cpu_count()
p = psutil.Process()
p.cpu_affinity([0,1,2,3])

from helpers import rl_covmat_ests_for_dataset as estimators
from helpers import helper_functions as hf
from helpers import eval_funcs_multi_target
from helpers import eval_funcs

from ONE_YR.NonLinear_Shrinkage import regression_evaluation_funcs as re_hf
from ONE_YR.NonLinear_Shrinkage import helper_functions_NL_RL as NL_hf

from collections import Counter

In [2]:
def get_ann_vola(return_df):
    return round(return_df.std() * np.sqrt(252) * 100, 3)

def get_ann_ret(return_df):
    return round(return_df.mean() * 252 * 100, 3)

In [3]:
# define factors
all_factors = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8,
               1.9, 2.0]
# num eigenvalues to modify
num_eigenvalues = [1, 5, 10, 25, 50]

'''
Description:
all_res is the 21 day lead std dev of the returns of the minvar portfolio
all_rawres is the return of each day
'''
# load min signals and rawres
cvc_grid_data = r"H:\all\RL_Shrinkage_2024\ONE_YR\preprocessing\cvc_shrinkages_grid_data"
all_res = pd.read_csv(cvc_grid_data + f"\\CVC_grid_allres.csv")
all_rawres = pd.read_csv(cvc_grid_data + f"\\CVC_grid_all_rawres.csv")

allres_min_idxes_full = all_res.idxmin(axis=1)[: -21].values
allres_min_idxes_full = np.insert(allres_min_idxes_full, 0, np.repeat(["1.0"], 21))

# for sanity check: BIASED version should generally be better than
# non biased version as it is literally the minimum over the future 21 days
# so using it as a signal should outperform
allres_min_idxes_BIASED = all_res.idxmin(axis=1).values
allres_min_idxes_BIASED = allres_min_idxes_BIASED


# simple argmin rule, with full allres_min, should be same results as above
allres_min_idxes_full_v2 = allres_min_idxes_full[list(range(0, allres_min_idxes_full.shape[0], 21))]
allres_min_idxes_full_v2 = np.repeat(allres_min_idxes_full_v2, 21)
res_simple_argmin_rule = np.diag(all_rawres.loc[:, allres_min_idxes_full_v2])[5040:]



# simple argmin rule, biased (as a sanity check)
allres_min_idxes_BIASED_v2 = allres_min_idxes_BIASED[list(range(0, allres_min_idxes_BIASED.shape[0], 21))]
allres_min_idxes_BIASED_v2 = np.repeat(allres_min_idxes_BIASED_v2, 21)
res_simple_argmin_rule_biased = np.diag(all_rawres.loc[:, allres_min_idxes_BIASED_v2])[5040:]



res_actual_argmin = []
for i in range(5313//21):
    tmp_data = all_rawres.iloc[5040 + 21*i: 5040 + 21*(i+1)]
    curmin_idx = tmp_data.std().idxmin()
    curmin = tmp_data.loc[:, curmin_idx]
    res_actual_argmin += curmin.tolist()
res_actual_argmin = np.array(res_actual_argmin)
# np.std(res_actual_argmin) * np.sqrt(252) * 100  --> 10.375

res_actual_argmin_nonbiased = []
for i in range(5313//21):
    idx_min_data = all_rawres.iloc[5040 - 21 + 21*i: 5040 - 21 + 21*(i+1)]
    curmin_idx = idx_min_data.std().idxmin()
    tmp_data = all_rawres.iloc[5040 + 21*i: 5040 + 21*(i+1)]
    curmin = tmp_data.loc[:, curmin_idx]
    res_actual_argmin_nonbiased += curmin.tolist()
res_actual_argmin_nonbiased = np.array(res_actual_argmin_nonbiased)

model_path = r"H:\all\RL_Shrinkage_2024\ONE_YR\Linear_Shrinkage\results"
lgbm_returns = pd.read_csv(model_path + "\\lgbm_cvc_model_returns.csv")
lgbm_returns = lgbm_returns.values

## Explanation of the Data used for training and evaluating the model

In the whole dataset we have roughly 10'000 trading days. Theoretically we could build a Minimum Variance portfolio **every** trading day, and hold it for 21 days. This gives us 21 days of return data, which we then use to calculate the annualized volatility of said portfolio.
Hence, the argmin signal of rebalancing date *i* is the Factor (in {0.5,...,2.0}), that would have led to the MVP with the lowest volatility over the next 21 trading days. Hence, this is a simple autoregressive signal. Of course, we lag it by 21 days, in order not to peek in the future. We do not use the signal itself as a input in our feature matrix, but the volatility.

Input Matrix X is given by:
- Volatility of MVP built by using optimal factor on CVC/QIS.
- Yearly and Monthly Volatility EW portfolio (vola of the EW PF daily returns) of currently included stocks
- Trace of sample covmat
- Yearly and Monthly Vola as well as Yearly and Monthly Timeseries Momentum for the current stock universe
- Mean of pairwise correlations of current stock universe returns
This results in a $X \ in (n \times 9)$ Input Matrix. We can also do a second version including the factor data (13 factor features).


The Y signal we use exaclty the same but without a lag of course. I.e., we predict the OOS volatility of each of the MVP built using each of the factors (I.e., we fit a 16 **separate** models for each of the factors, using the same input data, and pick the factor which leads the lowest predicted OOS volatility).
- $Y \in (n \times 16)$  

Hence, as the Y signals during training are 21 days forward looking, we always have a gap of 21 days between train and test sets. This gap persists for every rebalancing date.


Note to self: we still cannot use "opt_values" in our feature matrix, of course. As it would induce forward looking bias in "x_test". We actually can, as the "opt_values" are now also lagged by 21 days. I.e., x_test[i] containts Y.loc[i-21, min_idx]. The train and test sets still must be lagged by 21 days, as y_train contains 21 day forward looking data.

## Results of current model
The current model is a light GBM model and uses **the same** inputs in the Linear (CVC) model and the NonLinear (QIS) model. Now I have modified it to be the same procedure.
In the linear case this is: Given a CVC shrinkage intensity, apply a factor in {0.5, 0.6, ..., 1.9, 2.0} to the CVC shrinkage intensity. The factor is the actual learnable "action" of our RL agent.

Note that the actual argmin is the actual obtainable minimum when working with the original CVC shrinkage intensity times the above factor and rebalancing every 21 days, i.e., every 21 days we might change the factor for the rebalancing.



In [4]:
names = ["lgbm_cvc_model", "simple_argmin_rule", "simple_argmin_rule_biased", "actual_argmin"]
volas = [get_ann_vola(lgbm_returns), get_ann_vola(res_simple_argmin_rule), 
         get_ann_vola(res_simple_argmin_rule_biased), get_ann_vola(res_actual_argmin)]
rets = [get_ann_ret(lgbm_returns), get_ann_ret(res_simple_argmin_rule), 
        get_ann_ret(res_simple_argmin_rule_biased), get_ann_ret(res_actual_argmin)]

volas = pd.DataFrame(volas, columns=["Ann. Volas."], index=names)
rets = pd.DataFrame(rets, columns=["Ann. Rets"], index=names)

pd.concat([volas, rets], axis=1)

,Ann. Volas.,Ann. Rets
lgbm_cvc_model,10.531,10.083
simple_argmin_rule,10.645,9.668
simple_argmin_rule_biased,10.375,9.830
actual_argmin,10.375,9.830
